In [1]:
import pandas as pd
import seaborn as sns
from statsmodels.stats.inter_rater import fleiss_kappa, aggregate_raters

# Grade Analysis

In [2]:
grade_df = pd.read_csv('Menagerie/data/grades.csv', index_col=0)
grade_df

,assignment_number,comments,skill,participant_id,batch,grade
0,18.0,The code use meaningful identifier names and a...,Readability,15,1,B+
1,620.0,Code is well organized into logical classes an...,Readability,15,1,A+
2,466.0,"The code is well-organized, consistently inden...",Readability,15,1,A-
3,591.0,The code is well organized and readable. Inden...,Readability,15,1,A-
4,587.0,The code is well organized and readable. Inden...,Readability,15,1,A-
...,...,...,...,...,...,...
15,58.0,NaN,Documentation,14,1,A++
16,353.0,NaN,Documentation,14,1,A++
17,422.0,NaN,Documentation,14,1,A++
18,480.0,NaN,Documentation,14,1,A++


In [3]:
grade_df = grade_df[~grade_df['grade'].isnull()]
grade_df

,assignment_number,comments,skill,participant_id,batch,grade
0,18.0,The code use meaningful identifier names and a...,Readability,15,1,B+
1,620.0,Code is well organized into logical classes an...,Readability,15,1,A+
2,466.0,"The code is well-organized, consistently inden...",Readability,15,1,A-
3,591.0,The code is well organized and readable. Inden...,Readability,15,1,A-
4,587.0,The code is well organized and readable. Inden...,Readability,15,1,A-
...,...,...,...,...,...,...
15,58.0,NaN,Documentation,14,1,A++
16,353.0,NaN,Documentation,14,1,A++
17,422.0,NaN,Documentation,14,1,A++
18,480.0,NaN,Documentation,14,1,A++


## Inter-Rater Analysis

In [4]:
# TODO Discuss with Neil what to do with this mistake
# 105 duplicated accidentally between the two batches... in additional to the purposeful duplication.
skill_df = grade_df[grade_df['skill'] == 'Readability'][['participant_id', 'assignment_number', 'grade']]
group_df = skill_df[skill_df['participant_id'].isin([21, 22, 23, 24])].sort_values(by='participant_id')
group_df[group_df['assignment_number'] == 105]
# pivot = group_df.pivot(index='assignment_number', columns='participant_id', values='grade')

,participant_id,assignment_number,grade
4,21,105.0,A+
8,21,105.0,A+
4,22,105.0,A
8,22,105.0,A
4,23,105.0,B
8,23,105.0,C-
4,24,105.0,D+
8,24,105.0,B-


In [5]:
def calculate_fleiss_kappa(skill, participant_list):
    skill_df = grade_df[grade_df['skill'] == skill][['participant_id', 'assignment_number', 'grade']]
    group_df = skill_df[skill_df['participant_id'].isin(participant_list)].sort_values(by='participant_id')
    pivot = group_df.pivot(index='assignment_number', columns='participant_id', values='grade')
    if any(pivot.isna().any(axis=1)):
        print('Missing Grade: ', pivot[pivot.isna().any(axis=1)])
        pivot.dropna(inplace=True)
  
    return fleiss_kappa(aggregate_raters(pivot)[0], 'fleiss')

In [8]:
results_df = None

for skill in ['Correctness', 'Readability', 'Code Elegance', 'Documentation']:
    for i in range(1, 28, 4):
        group = list(range(i, i + 4))
        group_str = str(group[0]) + '-' + str(group[-1])
        try:
            results_df = pd.concat([results_df, pd.DataFrame({
                                    'group': group_str,
                                    'skill': skill,
                                    'fliess_kappa': calculate_fleiss_kappa(skill, group)}, index=[0])])
        except ValueError:
            print('Error duplicate entries check data for: ', group_str)
        
        
results_df

Missing Grade:  participant_id      13  14  15 16
assignment_number                
306.0              NaN  A+  B+  A
Error duplicate entries check data for:  21-24
Error duplicate entries check data for:  21-24
Error duplicate entries check data for:  21-24
Missing Grade:  participant_id      9  10  11  12
assignment_number                
117.0              NaN  B  A+  A-
Error duplicate entries check data for:  21-24
Missing Grade:  participant_id      25   26   27   28
assignment_number                    
394.0              A++  A++  NaN  A++


,group,skill,fliess_kappa
0,1-4,Correctness,0.042673
0,5-8,Correctness,-0.024104
0,9-12,Correctness,0.003440
0,13-16,Correctness,0.041807
0,17-20,Correctness,0.113360
0,25-28,Correctness,0.046496
0,1-4,Readability,-0.002874
0,5-8,Readability,-0.070433
0,9-12,Readability,-0.069315
0,13-16,Readability,-0.057512
